**DataTypes in PySpark:**  

RDD:
* Records

DataFrames:
* Rows
* Columns

Groupby:
* Groups

DataSets:


**F.struct in PySpark:**  
`pyspark.sql.functions.`**`struct`**`(*cols)`

Creates a new struct column.  
**Parameters:**	**cols** – list of column names (string) or list of **Column** expressions

```python
df = spark.createDataFrame([('Alice', 2), ('Bob', 5)], ['name','age'])

df.select(struct('age', 'name').alias("struct")).collect()
[Row(struct=Row(age=2, name=u'Alice')), Row(struct=Row(age=5, name=u'Bob'))]

df.select(struct([df.age, df.name]).alias("struct")).collect()
[Row(struct=Row(age=2, name=u'Alice')), Row(struct=Row(age=5, name=u'Bob'))]
```